In [1]:
import pandas as pd
import numpy as np
from elasticsearch import Elasticsearch
import re

import sys
sys.path.append('functions')
import preprocessing_fncs as ppf
import elastic_search_fncs as esf

# Details of the dataset
db_host = 'https://athena.london.gov.uk'
db_user = 'odbc_readonly'
db_pass = 'odbc_readonly'
db_port = '10099'
db_name = 'gla-ldd-external'

# Creates connection to the dataset
es = Elasticsearch(
    [f"{db_host}:{db_port}"],
    basic_auth=(db_user, db_pass),
    verify_certs=True,
    ca_certs='athena_es_full_chain.crt'
)

# Check connection
if es.ping():
    print("Connected to Elasticsearch!")
else:
    print("Could not connect to Elasticsearch.")

Connected to Elasticsearch!


In [2]:
import pandas as pd

def get_residential_units(es, year, index="applications"):
    """
    Retrieve specified columns for all entries where total_no_proposed_residential_units >= 1
    and decision_date is within a given year.

    Args:
        es (obj): Elasticsearch connection object.
        year (int): The year for which decision_date should be filtered.
        index (str, optional): Elasticsearch index name. Defaults to "applications".

    Returns:
        DataFrame: A Pandas DataFrame with the filtered applications.
    """
    
    query = {
        "query": {
            "bool": {
                "must": [
                    {
                        "range": {
                            "application_details.residential_details.total_no_existing_residential_units": {
                                "gte": 1
                            }
                        }
                    },
                    {
                        "range": {
                            "decision_date": {
                                "gte": f"01/01/{year}",
                                "lt": f"01/01/{year + 1}"
                            }
                        }
                    }
                ]
            }
        },
        "_source": [
            "lpa_app_no",
            "valid_date",
            "decision_date",
            "borough",
            "application_details.residential_details.total_no_existing_residential_units",
            "application_details.residential_details.total_no_affordable_units",
            "application_details.residential_details.site_area",
            "application_details.residential_details.habitable_rooms_density",
            "pp_id",
            "uprn",
            "status",
            "decision",
            "street_name",
            "site_name",
            "site_number",
            "polygon",
            "wgs84_polygon",
            "description"
        ]
    }
    
    # Initialize scrolling
    response = es.search(index=index, body=query, scroll="2m", size=10000)
    scroll_id = response['_scroll_id']
    hits = response['hits']['hits']
    
    all_hits = []
    all_hits.extend(hits)

    # Keep scrolling until no hits are returned
    while len(hits) > 0:
        response = es.scroll(scroll_id=scroll_id, scroll="2m")
        scroll_id = response['_scroll_id']
        hits = response['hits']['hits']
        all_hits.extend(hits)

    # Normalize the results into a DataFrame
    df = pd.json_normalize(all_hits)
    
    return df


In [ ]:
year = 2021

df = get_residential_units(es=es, year=year)
df = ppf.format_df(df)

C:\Users\hp\AppData\Local\Temp\ipykernel_59284\1004643928.py:62: DeprecationWarning: Received 'size' via a specific parameter in the presence of a 'body' parameter, which is deprecated and will be removed in a future version. Instead, use only 'body' or only specific parameters.
  response = es.search(index=index, body=query, scroll="2m", size=10000)


In [5]:
len(df)

2849